# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import base64
from config import Config

import os

import requests
from IPython import get_ipython
from IPython.display import display, Markdown

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
set time zone 'UTC';

In [ ]:
from datetime import datetime, timezone, timedelta
from dateutil.relativedelta import relativedelta
import pytz


def get_utc_timezone_adjusted_date(target_date) -> str:
    target_datetime = datetime.combine(target_date, datetime.min.time())
    hours = target_datetime.astimezone(pytz.timezone("America/Vancouver")).utcoffset().total_seconds() / 60 / 60
    target_date = target_datetime.replace(tzinfo=timezone.utc) + relativedelta(hours=-hours)
    return target_date.replace(tzinfo=None).strftime("%Y-%m-%d %H:%M:%S")


def get_first_last_month_dates_in_utc() -> tuple[str, str]:
    current_time = datetime.now(pytz.timezone("America/Vancouver"))
    last_month = current_time - relativedelta(months=1)
    from_date = last_month.replace(day=1)
    from_date = get_utc_timezone_adjusted_date(from_date)
    to_date = (last_month.replace(day=1) + relativedelta(months=1))
    to_date = get_utc_timezone_adjusted_date(to_date)
    return from_date, to_date


from_date, to_date = get_first_last_month_dates_in_utc()
print(f"Using from_date: {from_date} and to_date: {to_date}")

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT
    i.id as transaction_id,
    (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date_pacific,
    created_name,
    pa.auth_account_id as account_number,
    pa.name as account_name,
    total * CASE WHEN invoice_status_code = 'CANCELLED' THEN 0 ELSE 1 END as total,
    service_fees,
    total - service_fees as subtotal,
    payment_method_code,
    corp_type_code,
    payment_date,
    refund_date,
    invoice_status_code,
    folio_number,
    (select string_agg(quantity || 'x - ' || filing_type_code || ' - ' || description || ' - $' || pli.total, ',') from payment_line_items pli JOIN fee_schedules fs on fs.fee_schedule_id = pli.fee_schedule_id where invoice_id = i.id) as payment_line_items
FROM
    invoices i
LEFT JOIN
    payment_accounts pa ON i.payment_account_id = pa.id
WHERE
    corp_type_code = :partner_code
    AND total > 0
    AND invoice_status_code in ('PAID', 'CREDITED', 'REFUNDED', 'CANCELLED', 'REFUND_REQUESTED')
    AND payment_method_code in ('PAD','EJV', 'EFT', 'DIRECT_PAY', 'ONLINE_BANKING') 
    AND ((created_on >= :from_date and created_on <= :to_date) 
         OR (refund_date >= :from_date and refund_date <= :to_date))
ORDER BY
    1;

Save to CSV

In [ ]:
filename_summary = (
    os.path.join(os.getcwd(), r"data/")
    + partner_code
    + "_monthly_reconciliation_summary_"
    + datetime.strftime(datetime.now() - timedelta(1), "%Y-%m")
    + ".csv"
)
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, "w") as f:
    f.write("Monthly Reconciliation Summary:\n\n")
    if df_summary.empty:
        f.write("No Data Retrieved")
    else:
        df_summary.to_csv(f, sep=",", encoding="utf-8", index=False)

disbursement summary

In [ ]:
# Define the partner codes for which the disbursement summary should be executed
partners_for_disbursement_summary = Config.PARTNER_CODES_DISBURSEMENT.split(",")

print(f"Processing for partner_code: {partner_code}")
print(f"Partners for disbursement summary: {partners_for_disbursement_summary}")

# Only date part, no time saved for this field, also I believe this is pacific, it comes straight
# from the feedback files.
from_date = from_date.split(' ')[0] 
to_date = to_date.split(' ')[0]
print(f'Using from_date: {from_date} and to_date: {to_date}')
if partner_code in partners_for_disbursement_summary:
    print(f"Partner code {partner_code} found in the list, executing SQL query.")
    query = f"""
    SELECT 
        i.id as transaction_id,
        (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date_pacific,
        created_name,
        pa.auth_account_id as account_number,
        pa.name as account_name,
        total,
        service_fees,
        total - service_fees as disbursed_amount,
        payment_method_code,
        corp_type_code,
        payment_date,
        refund_date,
        invoice_status_code,
        folio_number,
        disbursement_date::date as disbursement_date_pacific,
        disbursement_reversal_date::date disbursement_reversal_date_pacific,
        disbursement_status_code,
        (select string_agg(quantity || 'x - ' || filing_type_code || ' - ' || description || ' - $' || pli.total , ',') from payment_line_items pli join fee_schedules fs on fs.fee_schedule_id = pli.fee_schedule_id where invoice_id = i.id) as payment_line_items
    FROM 
        invoices i
    LEFT JOIN
        payment_accounts pa ON i.payment_account_id = pa.id
    WHERE corp_type_code = :partner_code
    AND invoice_status_code in ('PAID', 'CREDITED', 'REFUNDED')
    AND payment_method_code in ('PAD','EJV', 'EFT', 'DIRECT_PAY', 'ONLINE_BANKING')
    AND ((disbursement_date >= :from_date AND disbursement_date <= :to_date)
        OR (disbursement_reversal_date >= :from_date AND disbursement_reversal_date <= :to_date))
    order by 1;
    """

    display(Markdown(f"## Running query for partner: {partner_code}"))
    results = get_ipython().run_cell_magic('sql', '', query)
    monthly_reconciliation_disbursed = results.DataFrame()  # Convert the results to a DataFrame for later use
else:
    print(f"Partner code {partner_code} not in the list, skipping SQL query.")
    monthly_reconciliation_disbursed = None

Save to another CSV

In [ ]:
if partner_code in partners_for_disbursement_summary:
    filename_disbursed = os.path.join(
        os.getcwd(),
        "data",
        f"{partner_code}_monthly_reconciliation_disbursed_"
        + datetime.strftime(datetime.now() - timedelta(1), "%Y-%m")
        + ".csv",
    )
    print(f"Saving CSV for partner_code: {partner_code} at {filename_disbursed}")
    with open(filename_disbursed, "w") as f:
        f.write("Monthly Reconciliation Disbursed:\n\n")
        if monthly_reconciliation_disbursed is None or monthly_reconciliation_disbursed.empty:
            f.write("No Data Retrieved")
        else:
            monthly_reconciliation_disbursed.to_csv(f, sep=",", encoding="utf-8", index=False)

Authenticate

In [ ]:
payload = "grant_type=client_credentials"
basic_hash = base64.b64encode(f"{Config.NOTEBOOK_SERVICE_ACCOUNT_ID}:{Config.NOTEBOOK_SERVICE_ACCOUNT_SECRET}".encode())

headers = {"Content-Type": "application/x-www-form-urlencoded", "Authorization": f"Basic {basic_hash.decode()}"}
response = requests.request(
    "POST", f"{Config.JWT_OIDC_ISSUER}/protocol/openid-connect/token", headers=headers, data=payload
)

assert response.status_code == 200
notebook_service_account_token = response.json().get("access_token")

In [ ]:

partner_details = {
    "CSO": {
        "companyName": "Ministry of Justice",
        "addressLine1": "PO Box 9249, Stn Prov Govt",
        "addressLine2": "6th Floor, 850 Burdett Avenue",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9J2"
    },
    "VS": {
        "companyName": "Vital Statistics Agency",
        "addressLine1": "PO Box 9657, Stn Prov Govt",
        "addressLine2": "",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9P3"
    },
    "RPT": {
        "companyName": "Property Taxation Branch",
        "addressLine1": "Ministry of Provincial Revenue",
        "addressLine2": "4th Floor, 1802 Douglas Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8T 4K6"
    },
    "ESRA": {
        "companyName": "Site Remediation Program, Authorizations and Remediation Branch",
        "addressLine1": "Ministry of Environment and Parks",
        "addressLine2": "525 Superior Street, 3rd floor",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8V 0C5"
    },
    "STRR": {
        "companyName": "Ministry of Housing and Municipal Affairs",
        "addressLine1": "PO BOX 9844, STN PROV GOVT",
        "addressLine2": "4th Floor, 614 Humboldt Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 1A4"
    }
}

def generate_report(partner_code):
    API_BASE_URL = Config.REPORT_API_URL + '/reports'
    if not Config.REPORT_API_URL:
        raise ValueError("The REPORT_API_URL environment variable is not set or is empty")

    url = API_BASE_URL
    headers = {
        'Authorization': f'Bearer {notebook_service_account_token}',
        'Content-Type': 'application/json',
        'Accept': 'application/pdf'
    }

    revenue_query = """
        SELECT
            COUNT(*) AS transaction_count,
            SUM(total) AS total_revenue,
            TO_CHAR(:from_date ::TIMESTAMP, 'Month') as month,
            corp_type_code as registry
        FROM
            invoices
        WHERE
            corp_type_code = :partner_code
            AND invoice_status_code = 'PAID'
            AND payment_method_code IN ('PAD', 'EJV', 'EFT', 'DIRECT_PAY', 'ONLINE_BANKING')
            AND (created_on >= :from_date AND created_on <= :to_date)
        GROUP BY
            corp_type_code
    """
    result = get_ipython().run_cell_magic('sql', '', revenue_query)
    disbursed_query = """
        SELECT
            SUM(total-service_fees) as disbursement,
            TO_CHAR(:from_date ::TIMESTAMP, 'Month') as month,
            corp_type_code as registry
        FROM
            invoices
        WHERE
            corp_type_code = :partner_code
            AND payment_method_code in ('PAD','EJV','EFT','DIRECT_PAY', 'ONLINE_BANKING')
            AND (disbursement_date >= :from_date AND disbursement_date <= :to_date)
            AND disbursement_status_code = 'COMPLETED'
        GROUP BY
            corp_type_code
    """
    result_disbursed = get_ipython().run_cell_magic('sql', '', disbursed_query)
    table_rows = [
        {
            'registry': partner_code,
            'transactionCounts': 0,
            'totalPayment': 0,
            'totalDisbursement': 0,
            'month': datetime.strptime(from_date, "%Y-%m-%d").strftime("%B")
        }
    ]
    if result:
        df = result.DataFrame()
        df_disbursed = result_disbursed.DataFrame()
        format_currency = lambda x: f"$ {x:,.2f}"
        format_number = lambda x: f"{x:,.0f}"
        df = df.assign(
            totalPayment=df.get('total_revenue', 0).map(format_currency),
            transactionCounts=df.get('transaction_count', 0).map(format_number),
            totalDisbursement=0,
        )
        if not df_disbursed.empty:
            df = df.assign(totalDisbursement=df_disbursed.get('disbursement', 0).map(format_currency))
        table_rows = df.to_dict(orient='records')

    current_date = datetime.now(tz=timezone.utc).strftime("%B %d, %Y")

    details = partner_details.get(partner_code, {})
    if not details:
        raise ValueError(f"No details found for partner code: {partner_code}")

    # Define the request body
    data = {
        "templateVars": {
            "date": current_date,
            "companyName": details["companyName"],
            "addressLine1": details["addressLine1"],
            "addressLine2": details["addressLine2"],
            "city": details["city"],
            "province": details["province"],
            "areaCode": details["areaCode"],
            "firstName": partner_code,
            "enterMonth": table_rows[0]['month'],
            "tableRows": table_rows
        },
        "templateName": "revenue_letter",
        "reportName": "revenue_letter"
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        pdf_content = response.content

        pdf_filename = os.path.join(os.getcwd(), 'data', f'{partner_code}_revenue_letter.pdf')
        with open(pdf_filename, 'wb') as pdf_file:
            pdf_file.write(pdf_content)

        print("PDF report saved successfully as 'payment_receipt.pdf'")
    else:
        print('Failed to get the report:', response.text)
generate_report(partner_code)